In [1]:
import os, sys, multiprocessing as mp, pandas as pd, requests, re
from bs4 import BeautifulSoup
pd.set_option('max_colwidth', 200)

In [2]:
r = requests.get('https://deckstats.net/decks/99839/957744-muldrotha-edh?lng=en')
r.content
soup = BeautifulSoup(r.content, 'lxml')

html_data = soup.find_all('div', {'id': 'deck_overview_cards'})

raw_data = str(html_data[0])
indices = [m.start() for m in re.finditer('target="_blank">', raw_data)]
card_list = []
for index in indices:
    card_text = raw_data[index+16:index+56]
    card_text = card_text[0:card_text.find(' </a>')].strip()
    card_list.append(card_text)
len(card_list) # len should be lower than total (duplicate basic lands)

103

In [2]:
cached_data = 'muldrotha.csv'
def get_card_data(card_name):
    r = requests.get('https://api.scryfall.com/cards/search/?q=' + card_name)
    return(r.json()) # we _should_ always get the card we need since titles are exact
card_data = []
if not os.path.exists(cached_data):
    pool = mp.Pool()
    card_data = pool.map(get_card_data, card_list)
    pool.close()

In [3]:
card_df = None
if os.path.exists(cached_data):
    card_df =pd.read_csv('muldrotha.csv')
else:
    card_df = pd.DataFrame(list(map(lambda x: x['data'][0], card_data)))
    card_df = card_df[['name','type_line','cmc','mana_cost','oracle_text','power','toughness','usd']]
    card_df.to_csv('muldrotha.csv', sep = ',', encoding = 'utf-8', index = False)
card_df.head()

,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
0,"Muldrotha, the Gravetide",Legendary Creature — Elemental Avatar,6.0,{3}{U}{B}{G},"During each of your turns, you may play up to one permanent card of each permanent type from your graveyard. (If a card has multiple permanent types, choose one as you play it.)",6,6,6.99
1,Alchemist's Refuge,Land,0.0,NaN,"{T}: Add {C} to your mana pool.\n{G}{U}, {T}: You may cast spells this turn as though they had flash.",NaN,NaN,2.06
2,Arcane Lighthouse,Land,0.0,NaN,"{T}: Add {C} to your mana pool.\n{1}, {T}: Until end of turn, creatures your opponents control lose hexproof and shroud and can't have hexproof or shroud.",NaN,NaN,2.79
3,Archfiend of Depravity,Creature — Demon,5.0,{3}{B}{B},"Flying\nAt the beginning of each opponent's end step, that player chooses up to two creatures he or she controls, then sacrifices the rest.",5,4,0.49
4,Ash Barrens,Land,0.0,NaN,"{T}: Add {C} to your mana pool.\nBasic landcycling {1} ({1}, Discard this card: Search your library for a basic land card, reveal it, put it into your hand, then shuffle your library.)",NaN,NaN,2.09


In [5]:
lands = card_df[card_df['type_line'].str.contains('Land', na = False)]
print('Land count: ' + str(len(lands)))
lands[['name','type_line','oracle_text','usd']].sort_values('usd', ascending=False)

Land count: 28


,name,type_line,oracle_text,usd
62,Overgrown Tomb,Land — Swamp Forest,"({T}: Add {B} or {G} to your mana pool.)\nAs Overgrown Tomb enters the battlefield, you may pay 2 life. If you don't, Overgrown Tomb enters the battlefield tapped.",9.96
94,Twilight Mire,Land,"{T}: Add {C} to your mana pool.\n{B/G}, {T}: Add {B}{B}, {B}{G}, or {G}{G} to your mana pool.",8.99
101,Woodland Cemetery,Land,Woodland Cemetery enters the battlefield tapped unless you control a Swamp or a Forest.\n{T}: Add {B} or {G} to your mana pool.,6.87
30,Flooded Grove,Land,"{T}: Add {C} to your mana pool.\n{G/U}, {T}: Add {G}{G}, {G}{U}, or {U}{U} to your mana pool.",6.48
43,Hinterland Harbor,Land,Hinterland Harbor enters the battlefield tapped unless you control a Forest or an Island.\n{T}: Add {G} or {U} to your mana pool.,5.99
27,Drowned Catacomb,Land,Drowned Catacomb enters the battlefield tapped unless you control an Island or a Swamp.\n{T}: Add {U} or {B} to your mana pool.,4.05
87,Sunken Ruins,Land,"{T}: Add {C} to your mana pool.\n{U/B}, {T}: Add {U}{U}, {U}{B}, or {B}{B} to your mana pool.",21.36
2,Arcane Lighthouse,Land,"{T}: Add {C} to your mana pool.\n{1}, {T}: Until end of turn, creatures your opponents control lose hexproof and shroud and can't have hexproof or shroud.",2.79
4,Ash Barrens,Land,"{T}: Add {C} to your mana pool.\nBasic landcycling {1} ({1}, Discard this card: Search your library for a basic land card, reveal it, put it into your hand, then shuffle your library.)",2.09
1,Alchemist's Refuge,Land,"{T}: Add {C} to your mana pool.\n{G}{U}, {T}: You may cast spells this turn as though they had flash.",2.06


In [6]:
mana_rocks = card_df[card_df['oracle_text'].str.contains('mana pool|Mana pool|{C}{C}', na = False)]
mana_rocks = mana_rocks[-mana_rocks['type_line'].str.contains('Land|land', na = False)]
print('Mana ramp count: ' + str(len(mana_rocks)))
mana_rocks.sort_values('usd', ascending=False)

Mana ramp count: 15


,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
35,Gilded Lotus,Artifact,5.0,{5},{T}: Add three mana of any one color to your mana pool.,NaN,NaN,8.99
7,Birds of Paradise,Creature — Bird,1.0,{G},Flying\n{T}: Add one mana of any color to your mana pool.,0,1,7.07
21,Deathrite Shaman,Creature — Elf Shaman,1.0,{B/G},"{T}: Exile target land card from a graveyard. Add one mana of any color to your mana pool.\n{B}, {T}: Exile target instant or sorcery card from a graveyard. Each opponent loses 2 life.\n{G}, {T}: ...",1,2,4.48
17,Cryptolith Rite,Enchantment,2.0,{1}{G},"Creatures you control have ""{T}: Add one mana of any color to your mana pool.""",NaN,NaN,2.99
26,Dimir Signet,Artifact,2.0,{2},"{1}, {T}: Add {U}{B} to your mana pool.",NaN,NaN,1.62
13,Commander's Sphere,Artifact,3.0,{3},{T}: Add to your mana pool one mana of any color in your commander's color identity.\nSacrifice Commander's Sphere: Draw a card.,NaN,NaN,0.99
42,Hedron Archive,Artifact,4.0,{4},"{T}: Add {C}{C} to your mana pool.\n{2}, {T}, Sacrifice Hedron Archive: Draw two cards.",NaN,NaN,0.57
56,Mind Stone,Artifact,2.0,{2},"{T}: Add {C} to your mana pool.\n{1}, {T}, Sacrifice Mind Stone: Draw a card.",NaN,NaN,0.33
78,Simic Signet,Artifact,2.0,{2},"{1}, {T}: Add {G}{U} to your mana pool.",NaN,NaN,0.25
38,Golgari Signet,Artifact,2.0,{2},"{1}, {T}: Add {B}{G} to your mana pool.",NaN,NaN,0.23


In [7]:
card_draw = card_df[card_df['oracle_text'].str.contains('Draw|draw', na = False)]
print('Card draw count: ' + str(len(card_draw)))
card_draw[['name','type_line','mana_cost','cmc','oracle_text','usd']].sort_values('cmc', ascending=False)

Card draw count: 15


,name,type_line,mana_cost,cmc,oracle_text,usd
57,Mulldrifter,Creature — Elemental,{4}{U},5.0,"Flying\nWhen Mulldrifter enters the battlefield, draw two cards.\nEvoke {2}{U} (You may cast this spell for its evoke cost. If you do, it's sacrificed when it enters the battlefield.)",1.77
42,Hedron Archive,Artifact,{4},4.0,"{T}: Add {C}{C} to your mana pool.\n{2}, {T}, Sacrifice Hedron Archive: Draw two cards.",0.57
48,"Kiora, the Crashing Wave",Legendary Planeswalker — Kiora,{2}{G}{U},4.0,"+1: Until your next turn, prevent all damage that would be dealt to and dealt by target permanent an opponent controls.\n−1: Draw a card. You may play an additional land this turn.\n−5: You get an...",2.30
13,Commander's Sphere,Artifact,{3},3.0,{T}: Add to your mana pool one mana of any color in your commander's color identity.\nSacrifice Commander's Sphere: Draw a card.,0.99
24,Dimir Cluestone,Artifact,{3},3.0,"{T}: Add {U} or {B} to your mana pool.\n{U}{B}, {T}, Sacrifice Dimir Cluestone: Draw a card.",0.08
52,Lorescale Coatl,Creature — Snake,{1}{G}{U},3.0,"Whenever you draw a card, you may put a +1/+1 counter on Lorescale Coatl.",0.09
71,Rhystic Study,Enchantment,{2}{U},3.0,"Whenever an opponent casts a spell, you may draw a card unless that player pays {1}.",NaN
73,Secrets of the Dead,Enchantment,{2}{U},3.0,"Whenever you cast a spell from your graveyard, draw a card.",0.11
84,Sultai Banner,Artifact,{3},3.0,"{T}: Add {B}, {G}, or {U} to your mana pool.\n{B}{G}{U}, {T}, Sacrifice Sultai Banner: Draw a card.",0.05
85,Sultai Charm,Instant,{B}{G}{U},3.0,"Choose one —\n• Destroy target monocolored creature.\n• Destroy target artifact or enchantment.\n• Draw two cards, then discard a card.",0.08


In [4]:
removal = card_df[card_df['oracle_text'].str.lower().str.contains('destroy target', na = False)]
removal = removal[-removal['type_line'].str.contains('Land')]
print('Targeted removal: ' + str(len(removal)))
removal # this one will require more thought

Targeted removal: 10


,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
6,Banewhip Punisher,Creature — Human Warrior,3.0,{2}{B},"When Banewhip Punisher enters the battlefield, you may put a -1/-1 counter on target creature.\n{B}, Sacrifice Banewhip Punisher: Destroy target creature that has a -1/-1 counter on it.",2,2,0.11
10,Caustic Caterpillar,Creature — Insect,1.0,{G},"{1}{G}, Sacrifice Caustic Caterpillar: Destroy target artifact or enchantment.",1,1,0.10
23,Dimir Charm,Instant,2.0,{U}{B},Choose one —\n• Counter target sorcery spell.\n• Destroy target creature with power 2 or less.\n• Look at the top three cards of target player's library. Put one back and the rest into that player...,NaN,NaN,0.10
37,Golgari Charm,Instant,2.0,{B}{G},Choose one —\n• All creatures get -1/-1 until end of turn.\n• Destroy target enchantment.\n• Regenerate each creature you control.,NaN,NaN,0.25
40,"Grimgrin, Corpse-Born",Legendary Creature — Zombie Warrior,5.0,{3}{U}{B},"Grimgrin, Corpse-Born enters the battlefield tapped and doesn't untap during your untap step.\nSacrifice another creature: Untap Grimgrin and put a +1/+1 counter on it.\nWhenever Grimgrin attacks,...",5,5,4.30
69,Putrefy,Instant,3.0,{1}{B}{G},Destroy target artifact or creature. It can't be regenerated.,NaN,NaN,0.22
72,Seal of Primordium,Enchantment,2.0,{1}{G},Sacrifice Seal of Primordium: Destroy target artifact or enchantment.,NaN,NaN,0.10
77,Shriekmaw,Creature — Elemental,5.0,{4}{B},"Fear (This creature can't be blocked except by artifact creatures and/or black creatures.)\nWhen Shriekmaw enters the battlefield, destroy target nonartifact, nonblack creature.\nEvoke {1}{B} (You...",3,2,1.11
81,Solemn Offering,Sorcery,3.0,{2}{W},Destroy target artifact or enchantment. You gain 4 life.,NaN,NaN,0.05
85,Sultai Charm,Instant,3.0,{B}{G}{U},"Choose one —\n• Destroy target monocolored creature.\n• Destroy target artifact or enchantment.\n• Draw two cards, then discard a card.",NaN,NaN,0.08
